<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Contrastive_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Data augmentation
transform = transforms.Compose([
    transforms.RandomResizedCrop(size=32),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)

# Basic encoder network
class Encoder(nn.Module):
    def __init__(self, feature_dim=128):
        super(Encoder, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Flatten()
        )
        self.fc = nn.Linear(128 * 8 * 8, feature_dim)

    def forward(self, x):
        x = self.conv(x)
        return self.fc(x)

# Contrastive loss function
def contrastive_loss(embeddings, temperature=0.5):
    normed_embeddings = nn.functional.normalize(embeddings, dim=1)
    similarity_matrix = torch.mm(normed_embeddings, normed_embeddings.t())
    labels = torch.arange(embeddings.size(0)).to(embeddings.device)
    labels = (labels.unsqueeze(0) == labels.unsqueeze(1)).float()
    loss = nn.CrossEntropyLoss()(similarity_matrix / temperature, labels)
    return loss

# Training
model = Encoder()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(100):
    for images, _ in train_loader:
        images = images.to('cuda')
        embeddings = model(images)
        loss = contrastive_loss(embeddings)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch}, Loss: {loss.item()}')

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Data augmentation
transform = transforms.Compose([
    transforms.RandomResizedCrop(size=32),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)

# Basic encoder network
class Encoder(nn.Module):
    def __init__(self, feature_dim=128):
        super(Encoder, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Flatten()
        )
        self.fc = nn.Linear(128 * 8 * 8, feature_dim)

    def forward(self, x):
        x = self.conv(x)
        return self.fc(x)

# Contrastive loss function
def contrastive_loss(embeddings, temperature=0.5):
    normed_embeddings = nn.functional.normalize(embeddings, dim=1)
    similarity_matrix = torch.mm(normed_embeddings, normed_embeddings.t())
    labels = torch.arange(embeddings.size(0)).to(embeddings.device)
    labels = (labels.unsqueeze(0) == labels.unsqueeze(1)).float()
    loss = nn.CrossEntropyLoss()(similarity_matrix / temperature, labels)
    return loss

# Training
model = Encoder()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(100):
    for images, _ in train_loader:
        embeddings = model(images)
        loss = contrastive_loss(embeddings)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch}, Loss: {loss.item()}')